# Deep Convolutional Neural Nets - CIFAR10

This notebook shows how to use a simple DCNN architecture to solve image classification problem from CIFAR10 dataset.

In [1]:
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt

/home/hcmarchezi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Create NN Model
The function below is used to create a DCNN model for the cifar10 problem.
Essentially the model can be described as:
* 5x5 conv 50
* 2x2 max pooling
* dense layer
* output layer

In [13]:
def create_model(input_shape, num_classes):
    model = Sequential()
    
    # 1st layer - convolutional layer: 5x5, 50 feature maps
    model.add(Conv2D(32, (3,3), padding="same", input_shape=input_shape))
    # RELU activation
    model.add(Activation("relu"))
    # 2nd layer - max pooling layer: 2x2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    #model.add(Conv2D(50, kernel_size=5, padding="same"))
    #model.add(Activation("relu"))
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    #model.add(Dropout(0.25, seed=100))   
    
    # 3rd layer - flatten previous layer - transform into a vector
    model.add(Flatten())
    # 4th layer - fully connected layer
    model.add(Dense(512))
    model.add(Activation("relu"))
    
    model.add(Dropout(0.5))
    
    # 5th layer - output (final) layer - softmax classifier
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    # Return constructed model
    return model

## Loading CIFAR10 Dataset
Loading training and test data from CIFAR10 dataset.

In [14]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Defining Constants
Defining constants based on CIFAR10 dataset. 60K 32x32 images with 3 channels. 10 labels.

In [15]:
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
NB_CLASSES = 10

## Data Preprocessing
Use one-hot-encoding for labels and normalization for X (/255).

In [16]:
# one hot encoding
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test  = np_utils.to_categorical(y_test,  NB_CLASSES)

# convert X matrices to float and divide for 255 (numbers between 0..1)
X_train = X_train.astype('float32') / 255
X_test  = X_test.astype('float32') / 255

## Train Model
Create model and describe it with summary.

In [17]:
model = create_model(input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS), num_classes=NB_CLASSES)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_9 (Activation)    (None, 512)               0         
__________

KeyboardInterrupt: 

## Evaluate Model
Evaluate trained model

In [10]:
score = model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("Test score:", score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 8s 803us/step
Test score: 1.2578526260375977
Test accuracy: 0.6691
